In [ ]:
# Subscribe NETPIE and save as CSV file

import paho.mqtt.client as mqtt
import socket
import csv
import json

MQTTbroker = 'mqtt.netpie.io'
id = 'b2bac658-4a03-496c-83e2-800e89ba36e4'
token = 'pENMaYNZq8J33CZNhdeZvp9UvHhXeG28'
secret = 'ujXd+TepnvrnY3tJ0rD#wL*obp-57$-u'
topic = '@msg/tgr2020/followUp/#'
fname = 'stage1.csv'

with open(fname, mode='w') as csv_file:
    fieldnames = ['gateway', 'timestamp', 'mac_addr', 'rssi', 'event_code']
    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    csv_writer.writeheader()
    
def on_connect(client, userdata, flags, rc):
    client.subscribe(topic)
    print('Subscribed')
    
def on_message(client, userdata, msg):
    print(msg.topic, ':', msg.payload)
    data = json.loads(msg.payload)
    words = msg.topic.split('/')
    data['gateway'] = words[-1]
    ieldnames = ['gateway', 'timestamp', 'mac_addr', 'rssi', 'event_code']
    with open(fname, mode='a') as csv_file:
        csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        csv_writer.writerow(data)

def on_disconnect(client, userdata, rc):
    print(userdata)
    
client = mqtt.Client(client_id=id)
client.username_pw_set(token)
client.connect(MQTTbroker)
client.on_connect = on_connect
client.on_message = on_message
client.on_disconnect = on_disconnect
client.socket().setsockopt(socket.SOL_SOCKET, socket.SO_SNDBUF, 2048)
client.loop_forever()

In [25]:
# Import CSV rawdata and convert as CSV file for training

import pandas as pd
import random

device = '3C:71:BF:44:F5:46'
stages = [0,1,2,3]
gateways = [1,2,3,4]
MIN_RSSI = -90

head = {'label':[], 'gw_1':[], 'gw_2':[], 'gw_3':[], 'gw_4':[]}
trainData = pd.DataFrame(head)

for stage in stages:
    fname = 'stage' + str(stage+1) + '.csv'
    data = pd.read_csv(fname)
    data.sort_values("timestamp", inplace=True) 
    data.drop_duplicates(inplace=True)
    print(data.head())
    
    for idx,row in data.iterrows():
        if row['mac_addr'] != device:
            continue
        newRow = {'label':stage, 'gw_1':MIN_RSSI+5*random.random(), 'gw_2':MIN_RSSI+5*random.random(), 'gw_3':MIN_RSSI+5*random.random(), 'gw_4':MIN_RSSI+5*random.random()}
        gw = row['gateway']
        key = 'gw_' + str(gw)
        newRow[key] = row['rssi']
        trainData = trainData.append( newRow, ignore_index=True)        
print(trainData.head())
trainData.to_csv('trainData.csv',index=False)

   gateway                   timestamp           mac_addr  rssi  event_code
0        1  2020-02-22 04:32:21.098892  3C:71:BF:44:F5:46   -62           0
2        1  2020-02-22 04:32:40.897909  3C:71:BF:44:F5:46   -58           0
4        1  2020-02-22 04:33:00.938606  3C:71:BF:44:F5:46   -55           0
6        1  2020-02-22 04:33:20.797358  3C:71:BF:44:F5:46   -61           0
   gateway                   timestamp           mac_addr  rssi  event_code
0        2  2020-02-22 04:38:42.399854  3C:71:BF:44:F5:46   -57           0
1        2  2020-02-22 04:39:02.260179  3C:71:BF:44:F5:46   -58           0
2        2  2020-02-22 04:39:22.056366  3C:71:BF:44:F5:46   -54           0
3        2  2020-02-22 04:39:42.037802  3C:71:BF:44:F5:46   -59           0
4        2  2020-02-22 04:40:01.900920  3C:71:BF:44:F5:46   -65           0
   gateway                   timestamp           mac_addr  rssi  event_code
0        3  2020-02-22 04:42:02.261781  3C:71:BF:44:F5:46   -55           0
1        3  

In [26]:
# Load dataset, train model, test classify
# https://www.tensorflow.org/tutorials/load_data/pandas_dataframe
# https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/

import pandas as pd
import tensorflow as tf
import numpy as np

# Load dataset
df = pd.read_csv('trainData.csv')
df['label'] = pd.Categorical(df['label'].astype(np.int32))
target = df.pop('label')
target = tf.one_hot(target, depth=4)
print(target)
MIN_RSSI = -90
df = df/MIN_RSSI
dataset = tf.data.Dataset.from_tensor_slices((df.values, target))
train_dataset = dataset.shuffle(len(df)).batch(2)
for feat, targ in train_dataset.take(4):
    print ('Features: {}, Target: {}'.format(feat, targ))

# train model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='tanh'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              #loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
loss = model.fit(train_dataset, epochs=5)

testdata = np.array([[-60.0, -90.0, -90.0, -90.0],
                    [-90.0, -50.0, -90.0, -90.0],
                    [-90.0, -90.0, -55.0, -90.0],
                    [-90.0, -90.0, -90.0, -50.0]])
testdata = testdata/MIN_RSSI
results = model.predict(testdata)
for r in results:
    print(r)


tf.Tensor(
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]], shape=(30, 4), dtype=float32)
Features: [[0.97138302 0.99149391 0.61111111 0.95046623]
 [0.99434989 0.97813983 0.97543175 0.63333333]], Target: [[0. 0. 1. 0.]
 [0. 0. 0. 1.]]
Features: [[0.68888889 0.98175048 0.94482785 0.96757793]
 [0.98814634 0.97784503 0.62222222 0.96439004]], Target: [[1. 0. 0. 0.]
 [0. 0. 1. 0.]]
Features: [[0.99055259 0.96824343 0.99849444 0.58888889]
 [0.96198044 0.73333333 0.94457975 0.94771102]], Target: [[0. 0. 0. 1.]
 [0. 1. 0. 0.]]
Features: [[0.9900227  0.71111111 0.97959521 0.94485628]
 [0.99921034 0.